In [39]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import soundfile as sf
from scipy.fft import rfft, rfftfreq
import librosa
import resampy
import librosa.display
import IPython.display as ipd
import os
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from pydub import AudioSegment
import os
warnings.filterwarnings('ignore')


In [40]:
def mfcc_feature_extractor(audio, sampleRate):
    mfccsFeatures = librosa.feature.mfcc(y=audio, sr=sampleRate, n_mfcc=40)
    mfccsScaledFeatures = np.mean(mfccsFeatures.T, axis=0)

    return mfccsScaledFeatures

In [41]:
def contrast_feature_extractor(audio, sampleRate):
    stft = np.abs(librosa.stft(audio))
    contrast = librosa.feature.spectral_contrast(S=stft, sr=sampleRate)
    contrastScaled = np.mean(contrast.T, axis=0)

    return contrastScaled

In [42]:
def tonnetz_feature_extractor(audio, sampleRate):
    tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(audio), sr=sampleRate)
    tonnetzScaled = np.mean(tonnetz.T, axis=0)

    return tonnetzScaled

In [43]:
def chroma_feature_extractor(audio,sampleRate):
    stft = np.abs(librosa.stft(audio))
    chroma = librosa.feature.chroma_stft(S=stft,sr=sampleRate)
    chromaScaled = np.mean(chroma.T,axis=0)
    
    return chromaScaled

In [44]:
def features_extractor(file):
    if file.lower().endswith(".m4a"):
        audio = AudioSegment.from_file(file, format="m4a")
        file_name, _ = os.path.splitext(os.path.basename(file))
        output_path = os.path.join(os.path.dirname(file), file_name + ".wav")
        audio.export(output_path, format="wav")
        os.remove(file)
        file = output_path

    features = []
    audio, sample_rate = librosa.load(file, res_type='kaiser_fast') 
    mfcc = mfcc_feature_extractor(audio, sample_rate)
    contrast = contrast_feature_extractor(audio, sample_rate)
    tonnetz = tonnetz_feature_extractor(audio, sample_rate)
    chroma = chroma_feature_extractor(audio, sample_rate)

    # Concatenate individual feature arrays
    concatenated_features = np.concatenate((mfcc, contrast, tonnetz, chroma), axis=0)
    features.append(concatenated_features)

    return features

In [58]:
def add_features(extractedFeatures, dirPath, label):
    for file in os.listdir(dirPath):
        filePath = os.path.join(dirPath, file)

        if filePath.lower().endswith(".m4a"):
            audio = AudioSegment.from_file(filePath, format="m4a")
            file_name, _ = os.path.splitext(os.path.basename(filePath))
            output_path = os.path.join(os.path.dirname(filePath), file_name + ".wav")
            audio.export(output_path, format="wav")
            os.remove(filePath)
            filePath = output_path

        audio, sampleRate = librosa.load(filePath, res_type='kaiser_fast') 
        mfcc = mfcc_feature_extractor(audio, sampleRate)
        contrast = contrast_feature_extractor(audio, sampleRate)
        tonnetz = tonnetz_feature_extractor(audio, sampleRate)
        chroma = chroma_feature_extractor(audio, sampleRate)

        extractedFeatures.append([mfcc, contrast, tonnetz, chroma, label])

In [89]:
dict1 = {
"open":[
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/ahmeda",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/ahmedk",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/hassan",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/hazem",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/ibrahim",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/mohannad",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/omar",
],

"unlock":[
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/ahmeda",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/ahmedk",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/hassan",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/hazem",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/ibrahim",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/mohannad",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/omar",
],

"grant":[
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/ahmeda",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/ahmedk",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/hassan",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/hazem",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/ibrahim",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/mohannad",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/omar",

],
"other":[
  "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/ahmeda",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/ahmedk",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/hassan",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/hazem",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/ibrahim",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/mohannad",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/omar",

]
}


In [90]:
extractedFeatures = []
for i in dict1.keys():
    for x in dict1[i]:
        add_features(extractedFeatures, x, i)

In [91]:
data = pd.DataFrame(extractedFeatures, columns=['mfcc','contrast','tonnetz','chroma','class'])
print(data.shape)
data.to_csv('password.csv', index=False)
data.describe().T

(316, 5)


,count,unique,top,freq
mfcc,316,316,"[-488.23578, 103.44409, 9.910593, 33.63195, 6....",1
contrast,316,316,"[23.98104457049824, 14.883558570433792, 19.139...",1
tonnetz,316,316,"[-0.01461258018698804, 0.0065578272256597275, ...",1
chroma,316,316,"[0.5946693, 0.6137347, 0.62787664, 0.614527, 0...",1
class,316,4,other,97


In [92]:
features = data.iloc[:, 0:-1]
target = data['class']
target.unique()

array(['open', 'unlock', 'grant', 'other'], dtype=object)

In [93]:
features = features.values.tolist()
for i in range(len(features)):
    features[i] = np.concatenate((features[i][0],features[i][1],features[i][2],features[i][3]))

In [94]:
encoder = LabelEncoder()
target = encoder.fit_transform(target)

In [118]:
class_counts = np.bincount(target)

for class_label, count in enumerate(class_counts):
    print(f"Class {class_label}: {count} instances")

Class 0: 71 instances
Class 1: 75 instances
Class 2: 97 instances
Class 3: 73 instances


In [96]:
xTrain, xTest, yTrain, yTest = train_test_split(features, target, test_size=0.25, random_state=105)

### creating RandomForest model

In [124]:
classifier = RandomForestClassifier(n_estimators=100, criterion="entropy")
classifier.fit(xTrain,yTrain)

RandomForestClassifier(criterion='entropy')

### creating SVM model

In [125]:
# classifier = SVC(kernel='linear',decision_function_shape="ovo" ,probability=True) 
# classifier.fit(xTrain,yTrain)

### creating XGBoost model

In [126]:
# classifier = XGBClassifier(objective='multi:softmax', num_class=3, eval_metric='mlogloss')
# classifier.fit(xTrain, yTrain)

In [127]:
preds = classifier.predict(xTest)

In [128]:
print(classification_report(yTest, preds))

              precision    recall  f1-score   support

           0       0.95      1.00      0.98        20
           1       0.93      0.82      0.87        17
           2       0.72      1.00      0.84        21
           3       0.93      0.62      0.74        21

    accuracy                           0.86        79
   macro avg       0.88      0.86      0.86        79
weighted avg       0.88      0.86      0.86        79



In [129]:
def prediction(pred):
    if pred == 0:
        print("grant")
    elif pred == 1:
        print("open")
    elif pred == 2:
        print("other")
    elif pred == 3:
        print("unlock")

In [130]:
testFeatures = features_extractor("C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/omar/omar_open_middle_door_1.wav") # open middle door
pred = classifier.predict(testFeatures)
prediction(pred)
print(classifier.predict_proba(testFeatures))

open
[[0.02 0.8  0.17 0.01]]


In [131]:
testFeatures = features_extractor("C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/mohannad/Recording (57).wav") # unlock the gate
pred = classifier.predict(testFeatures)
prediction(pred)
print(classifier.predict_proba(testFeatures))

unlock
[[0.09 0.04 0.02 0.85]]


In [132]:
testFeatures = features_extractor("C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/hazem/Recording (11).wav") # grant me access
pred = classifier.predict(testFeatures)
prediction(pred)
print(classifier.predict_proba(testFeatures))

grant
[[0.86 0.01 0.04 0.09]]


In [133]:
for file in os.listdir("C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/omar")[0:10]:
  testFeatures = features_extractor(f"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/omar/{file}") 
  pred = classifier.predict(testFeatures)
  prediction(pred) 
  print(classifier.predict_proba(testFeatures))

other
[[0.02 0.04 0.83 0.11]]
other
[[0.06 0.24 0.59 0.11]]
other
[[0.09 0.02 0.81 0.08]]
other
[[0.02 0.05 0.83 0.1 ]]
other
[[0.18 0.05 0.62 0.15]]
other
[[0.01 0.03 0.82 0.14]]
other
[[0.   0.09 0.91 0.  ]]
other
[[0.06 0.08 0.84 0.02]]
other
[[0.02 0.07 0.88 0.03]]
other
[[0.04 0.07 0.83 0.06]]


In [116]:
for file in os.listdir("C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/hazem")[0:5]:
  testFeatures = features_extractor(f"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/hazem/{file}") 
  pred = classifier.predict(testFeatures)
  prediction(pred)
  print(classifier.predict_proba(testFeatures))

open
[[0.00303704 0.98827946 0.00287284 0.00581058]]
open
[[0.00259669 0.9843907  0.00716484 0.00584781]]
open
[[0.00240238 0.9907414  0.00447072 0.00238548]]
open
[[6.0043263e-04 9.8312843e-01 1.2561117e-02 3.7099973e-03]]
other
[[0.08900943 0.09183308 0.697933   0.12122451]]


In [108]:
# for file in os.listdir("C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other people")[0:5]:
#   testFeatures = features_extractor(f"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other people/{file}") 
#   pred = classifier.predict(testFeatures)
#   prediction(pred)
#   print(classifier.predict_proba(testFeatures))

In [117]:
for file in os.listdir("C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/tamer")[0:5]:
  testFeatures = features_extractor(f"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/tamer/{file}") 
  pred = classifier.predict(testFeatures)
  prediction(pred)
  print(classifier.predict_proba(testFeatures))

other
[[0.00332559 0.00211169 0.9903696  0.00419311]]
other
[[2.1150447e-03 3.7611549e-04 9.9682659e-01 6.8222341e-04]]
other
[[1.0791096e-03 3.5145780e-04 9.9791855e-01 6.5088214e-04]]
other
[[4.7484101e-04 2.4371494e-04 9.9898118e-01 3.0025665e-04]]
other
[[7.4135291e-04 4.7120408e-04 9.9773705e-01 1.0504073e-03]]


In [110]:
pickle_out = open("password.pkl", "wb")
pickle.dump(classifier, pickle_out)
pickle_out.close()